In [ ]:
from keras.layers import LSTM, Embedding, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from keras import Input
from eval_utils import calculate_results, prepare_for_evaluation
from dataloader import *
from  config_translator import *
from data_processor import preprocess_data

In [ ]:
data = DataLoader(config_mixed_coco14_coco14_glove)

In [ ]:
data=preprocess_data(data)

In [ ]:
class ModelImpl:
    def __init__(self, data):
        self.data=data
        LSTM_NODES=256

        embedding_layer = Embedding(self.data.num_words_inputs,glove["embedings_dim"], weights=[self.data.embedding_matrix_input], input_length=self.data.max_input_len)
        encoder_inputs = Input(shape=(self.data.max_input_len,))
        x = embedding_layer(encoder_inputs)
        encoder = LSTM(LSTM_NODES, return_state=True)
        encoder_outputs, h, c = encoder(x)
        encoder_states = [h, c]
        # The next step is to define the decoder. The decoder will have two inputs: the hidden state and cell state from the encoder and the input sentence, which actually will be the output sentence with an <sos> token appended at the beginning.

        decoder_inputs = Input(shape=(self.data.max_output_len,))
        decoder_embedding = Embedding(self.data.num_words_output, LSTM_NODES)
        decoder_inputs_x = decoder_embedding(decoder_inputs)
        decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
        decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)
        # Finally, the output from the decoder LSTM is passed through a dense layer to predict decoder outputs.
        decoder_dense = Dense(self.data.num_words_output, activation='softmax')
        decoder_outputs = decoder_dense(decoder_outputs)
        self.encoder_model = Model([encoder_inputs,decoder_inputs], decoder_outputs)
        self.encoder_model.compile(
            optimizer=self.optimizer(),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        self.encoder_model.summary()

        def decoder_model(decoder_embedding,decoder_lstm,decoder_dense,LSTM_NODES=256):
            decoder_state_input_h = Input(shape=(LSTM_NODES,))
            decoder_state_input_c = Input(shape=(LSTM_NODES,))
            decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
            decoder_inputs_single = Input(shape=(1,))
            decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)
            decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)
            decoder_states = [h, c]
            decoder_outputs = decoder_dense(decoder_outputs)
            decoder_model = Model([decoder_inputs_single] + decoder_states_inputs, [decoder_outputs] + decoder_states)
            return decoder_model

        self.decoder_model = decoder_model(decoder_embedding,decoder_lstm,decoder_dense,LSTM_NODES=256)
        self.setup()

    def setup(self):
        # model.optimizer.lr = 0.001
        self.epochs = 100
        self.batch_size=64
        number_pics_per_bath = 3
        self.steps = len(self.data.encoder_input_sequences) // number_pics_per_bath

    def optimizer(self):
        return Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)



    def train(self):
        model_weights_path="./" + self.data.configuration["data_name"] + self.data.configuration["model_save_dir"]
        if self.data.configuration["train_model"]:
            es = EarlyStopping(monitor='loss', min_delta=0.001, patience=3)
            filepath = model_weights_path + self.data.configuration["model_save_path"]
            checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True,
                                         mode='min', save_weights_only=False)
            callbacks_list = [checkpoint, es, CSVLogger("./" + self.data.configuration["data_name"] +'/logs.csv', separator=",", append=True),]
            if self.data.configuration["continue_training"]:
                self.encoder_model = load_model(filepath)
                print("New model loaded")

            self.encoder_model.fit([self.data.encoder_input_sequences, self.data.decoder_input_sequences], self.data.decoder_targets_one_hot,
                epochs=self.epochs,
                steps_per_epoch=self.steps,
                callbacks=[callbacks_list],
                verbose=1,
            )
            if self.data.configuration["save_model"]:
                writepath = model_weights_path+ "/"+'model' + '.h5'
                self.encoder_model.save(writepath)
                self.encoder_model.save_weights(model_weights_path
                                        + self.data.configuration["model_save_path"])
        else:
            self.encoder_model.load_weights(model_weights_path
                                        +self.data.configuration["model_save_path"])

    def evaluate(self):
        expected, results = prepare_for_evaluation(self.data.encoder_test_sequences, self.data.test_captions_mapping,self.encoder_model, self.decoder_model, self.word2idx_outputs
                                                   , self.data.max_output_len)
        out = calculate_results(expected, results, self.data.configuration)
        print(out)
model=ModelImpl(data)

In [ ]:
model.train()